# 🏝️ AI Survivor Workshop

> **First step:** Click **"Copy to Drive"** in the toolbar above to save your own copy!

**Welcome!** In the next 2 hours, you'll go from zero to building your own AI agent that will compete against everyone else's in a live game of social strategy.

No coding experience required. Just run each cell in order and follow along.

### What We're Building Toward

By the end of this workshop, you'll have built an AI "contestant" with its own personality and strategy. All our agents will then be dropped into a shared game where they:

1. **Talk**
2. **Form alliances**
3. **Vote**

# Part 1: The Capability Ladder


### Setup

Run this cell first cell. It installs what we need and sets up the connection to the AI model.


In [ ]:
# Run this cell first!
%pip install anthropic --quiet

import json

import anthropic
from IPython.display import HTML, Markdown, display

# Workshop API key (provided by facilitator)
API_KEY = ""

client = anthropic.Anthropic(api_key=API_KEY)

MODEL = "claude-sonnet-4-20250514"

print("✅ Setup complete! You're ready to go.")

### Step 1: The Single LLM Call

The simplest thing you can do with an LLM is send a message in, get a response back.

```
Input  →  [LLM]  →  Output
```

That's it. One question, one answer.


In [ ]:
# A helper function that wraps the API call


def call_llm(prompt, system=None):
    """Send a message to the LLM and get a response back."""
    kwargs = {
        "model": MODEL,
        "max_tokens": 1024,
        "messages": [{"role": "user", "content": prompt}],
    }
    if system:
        kwargs["system"] = system
    response = client.messages.create(**kwargs)
    return response.content[0].text


# Let's try it!
result = call_llm(
    "Introduce yourself as a reality TV show contestant in 2-3 sentences."
)
print(result)

### 🎯 Exercise 1

Try changing the prompt below and running it. Ask the LLM something — anything!


In [ ]:
# Exercise 1: Change this prompt to whatever you want
my_prompt = ""  # <-- Type your prompt here

result = call_llm(my_prompt)
print(result)

### Step 2: Structured Output

Getting free-form text is nice, but what if we want **data** we can use in code?

We can ask the LLM to respond in a specific format (like JSON) so our program can parse and use the result.


In [ ]:
# Ask for structured data

prompt = """
Create a reality TV contestant profile. Respond ONLY with valid JSON, no other text:
{
    "name": "contestant name",
    "age": number,
    "personality": "2-3 word description",
    "strategy": "their game strategy in one sentence",
    "catchphrase": "their signature catchphrase"
}
"""

result = call_llm(prompt)
print("Raw response:")
print(result)
print("\n" + "=" * 50 + "\n")

# Parse it into actual data we can use
contestant = json.loads(result)
print(f"Name: {contestant['name']}")
print(f"Strategy: {contestant['strategy']}")
print(f'Catchphrase: "{contestant["catchphrase"]}"')

**Why this matters:** Now the LLM's output isn't just text — it's _data_ our code can work with.


### Step 3: Chained Calls (Workflows)

What if we need multiple steps? We can **chain** LLM calls together, where the output of one becomes the input to the next.

```
Input → [LLM Call 1] → result₁ → [LLM Call 2] → result₂ → [LLM Call 3] → Final Output
```

**Insight:** Your _code_ decides the order. The LLM just executes each step.


In [ ]:
# A workflow: Generate two contestants, then simulate their first interaction

print("⏳ Step 1: Creating Contestant A...")
contestant_a = call_llm(
    "Create a ruthless, strategic reality TV contestant. Give a name and one personality trait. Be brief, 1 sentence."
)
print(f"   → {contestant_a}\n")

print("⏳ Step 2: Creating Contestant B...")
contestant_b = call_llm(
    "Create a charming, social reality TV contestant. Give a name and one personality trait. Be brief, 1 sentence."
)
print(f"   → {contestant_b}\n")

print("⏳ Step 3: Simulating their first meeting...")
interaction = call_llm(f"""
Two reality TV contestants meet for the first time:
- Contestant A: {contestant_a}
- Contestant B: {contestant_b}

Write a brief 4-line dialogue of their first interaction.
""")

print("📺 THE SCENE:")
print(interaction)

### ⚠️ The Problem: Compounding Errors

There's a catch with workflows. If each step is 90% accurate:

| Steps | Overall Accuracy |
| ----- | ---------------- |
| 1     | 90%              |
| 2     | 81%              |
| 3     | 73%              |
| 4     | 66%              |
| 5     | 59%              |

**The formula:** `P(correct) = (1 - error_rate)^steps`

The more steps you chain, the more things can go wrong. This is the fundamental challenge of building with LLMs.


In [ ]:
# Visualizing compounding errors

print("If each step is 90% accurate:\n")
for steps in range(1, 8):
    accuracy = 0.90**steps
    bar = "█" * int(accuracy * 30)
    print(f"  {steps} steps: {accuracy * 100:5.1f}% {bar}")

### 🎯 Exercise 2

Build your own multi-step workflow. The topic is up to you.

Ideas:

- Step 1: Generate a character → Step 2: Generate a conflict → Step 3: Write a resolution
- Step 1: Pick an industry → Step 2: Identify a problem → Step 3: Propose a solution
- Or anything else!


In [ ]:
# Exercise 2: Build a 3-step workflow

# Step 1
step1 = call_llm("")  # <-- Your prompt here
print(f"Step 1: {step1}\n")

# Step 2 (use step1's output)
step2 = call_llm("")  # <-- Your prompt here, reference {step1}
print(f"Step 2: {step2}\n")

# Step 3 (use previous outputs)
step3 = call_llm("")  # <-- Your prompt here, reference {step1} and/or {step2}
print(f"Step 3: {step3}")

### Step 4: Tool Calling

So far, the LLM can only talk. But what if it could do things?

**Tool calling** lets us give the LLM a set of functions it can choose to invoke. The LLM decides _which_ tool to use and _what arguments_ to pass.

```
Input  →  [LLM]  →  "I want to use tool X with these arguments"  →  [Code runs tool]  →  Result
```

The LLM can't actually run the tools — it just tells us which one it wants. Our code executes it.


In [ ]:
# Define the tools the LLM can use
# These are the same tools our AI Survivor contestants will have

GAME_TOOLS = [
    {
        "name": "speak_publicly",
        "description": "Say something that ALL contestants can hear. Use this for public statements, accusations, or rallying support.",
        "input_schema": {
            "type": "object",
            "properties": {
                "message": {"type": "string", "description": "What to say publicly"}
            },
            "required": ["message"],
        },
    },
    {
        "name": "whisper_to",
        "description": "Send a private message to ONE specific contestant. No one else can hear this.",
        "input_schema": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "Name of the contestant to whisper to",
                },
                "message": {"type": "string", "description": "The private message"},
            },
            "required": ["recipient", "message"],
        },
    },
    {
        "name": "vote_to_eliminate",
        "description": "Cast your vote to eliminate a contestant. This is secret — no one sees who you voted for.",
        "input_schema": {
            "type": "object",
            "properties": {
                "target": {
                    "type": "string",
                    "description": "Name of the contestant to vote against",
                },
                "reason": {
                    "type": "string",
                    "description": "Your private reasoning (not shared)",
                },
            },
            "required": ["target"],
        },
    },
]

print("🔧 Available tools:")
for tool in GAME_TOOLS:
    print(f"   • {tool['name']}: {tool['description'][:60]}...")

In [ ]:
# Give the LLM a situation and let it CHOOSE which tool to use

scenario = """
You are "Marcus", a strategic contestant on a survival competition show.
Other contestants: Julia (friendly), Derek (aggressive), Sam (quiet).

The situation: You just overheard Derek planning to vote you out tonight.

What do you do? Use the available tools to take action.
"""

response = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    tools=GAME_TOOLS,
    messages=[{"role": "user", "content": scenario}],
)

# See what the LLM decided to do
print("🎬 THE LLM DECIDED TO:\n")
for block in response.content:
    if block.type == "tool_use":
        print(f"  📌 Tool: {block.name}")
        print(f"     Args: {json.dumps(block.input, indent=4)}")
        print()
    elif block.type == "text" and block.text.strip():
        print(f"  💭 Thinking: {block.text[:200]}\n")

**Note:** We didn't tell the LLM _which_ tool to use. It analyzed the situation and chose the right action on its own.

But we're still giving it one situation and getting one response. What if it could keep going?


## Step 5: The Agent Loop

Now we put it all together. An Agent is an LLM that:

1. **Observes** the current situation
2. **Thinks** about what to do
3. **Acts** using tools
4. **Repeats** until the goal is achieved

```
    ┌─────────┐
    │ OBSERVE │ ◄──────────┐
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │  THINK  │            │
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │   ACT   │────────────┘
    └─────────┘
```

The key difference from workflows:

- **Workflows:** Code decides the steps
- **Agents:** The LLM decides the steps


In [ ]:
# A simple agent that takes multiple actions


def run_agent(name, personality, goal, game_state, max_steps=3):
    """Run an agent that pursues a goal by choosing its own actions."""

    system_prompt = f"""
                    You are {name}, a contestant on AI Survivor, a reality competition.
                    Personality: {personality}
                    Your goal: {goal}
                    Be strategic. Stay in character. Take ONE action at a time.
                    """

    actions_taken = []

    for step in range(max_steps):
        print(f"\n--- Step {step + 1} ---")

        prompt = f"""
                CURRENT SITUATION:
                {game_state}

                YOUR PREVIOUS ACTIONS THIS ROUND:
                {json.dumps(actions_taken, indent=2) if actions_taken else "None yet."}

                What do you do next? Choose ONE action.
                """

        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system=system_prompt,
            tools=GAME_TOOLS,
            messages=[{"role": "user", "content": prompt}],
        )

        for block in response.content:
            if block.type == "tool_use":
                action = {"tool": block.name, "args": block.input}
                actions_taken.append(action)
                print(f"  🎬 {name} uses '{block.name}':")
                print(f"     {json.dumps(block.input)}")
            elif block.type == "text" and block.text.strip():
                print(f"  💭 {block.text[:150]}")

    return actions_taken

In [ ]:
# Let's run an agent and watch it decide its own actions!

game_state = """
            Contestants remaining: Marcus, Julia, Derek, Sam
            Recent events:
              - Derek won the immunity challenge
              - Julia has been talking to everyone privately
              - Sam hasn't spoken much all day
              - Tribal council is tonight. Someone is going home
            """

actions = run_agent(
    name="Marcus",
    personality="Strategic and calculating, but presents as friendly",
    goal="Survive tonight's vote. Build alliances. Don't be the target.",
    game_state=game_state,
    max_steps=3,
)

print("\n" + "=" * 50)
print("📋 ALL ACTIONS TAKEN:")
for i, action in enumerate(actions, 1):
    print(f"  {i}. {action['tool']}: {json.dumps(action['args'])}")

**Note:** We didn't tell the agent what to do but we told it _what to achieve_. It figured out the steps on its own.

|              | Workflows          | Agents            |
| ------------ | ------------------ | ----------------- |
| Who decides? | Code decides steps | LLM decides steps |
| Behavior     | Predictable        | Flexible          |
| Cost         | Cheaper & faster   | More expensive    |
| Best for     | Known paths        | Open-ended goals  |


# ☕ COFFEE BREAK

Take 15 minutes. When we come back, you'll build your own contestant.


# Part 2: Build Your Contestant

Now it's your turn. Fill in the details below to create your AI contestant.

This is the agent that will represent you in the game!


In [ ]:
# Give your contestant a name
CONTESTANT_NAME = ""

# Describe their personality (2-3 sentences)
PERSONALITY = ""  # e.g., "Ruthlessly strategic but hides it behind a warm smile."

# What's their strategy for winning?
STRATEGY = ""  # e.g., "Form a strong early alliance, then eliminate threats."

# Their signature catchphrase
CATCHPHRASE = ""  # e.g., "It's not personal, it's just math."

AGGRESSION = 5  # 1 = passive, 10 = attacks everyone

HONESTY = 5  # 1 = compulsive liar, 10 = cannot tell a lie

ALLIANCE_SEEKING = 5  # 1 = lone wolf, 10 = needs allies to function

PARANOIA = 5  # 1 = trusts everyone blindly, 10 = trusts no one

DRAMA = 5  # 1 = monotone robot, 10 = MAXIMUM DRAMA

In [ ]:
def validate():
    errors = []
    if not CONTESTANT_NAME:
        errors.append("❌ CONTESTANT_NAME is empty")
    if not PERSONALITY.strip():
        errors.append("❌ PERSONALITY is empty")
    if not STRATEGY.strip():
        errors.append("❌ STRATEGY is empty")
    if not CATCHPHRASE:
        errors.append("❌ CATCHPHRASE is empty")
    for label, val in [
        ("AGGRESSION", AGGRESSION),
        ("HONESTY", HONESTY),
        ("ALLIANCE_SEEKING", ALLIANCE_SEEKING),
        ("PARANOIA", PARANOIA),
        ("DRAMA", DRAMA),
    ]:
        if not (1 <= val <= 10):
            errors.append(f"❌ {label} must be between 1 and 10")
    if errors:
        print("\n".join(errors))
        return False

    def bar(v):
        return "█" * v + "░" * (10 - v)

    print(f"""
        ✅ CONTESTANT READY!

        ╔══════════════════════════════════════════════════╗
        ║  {CONTESTANT_NAME:^46}  ║
        ╚══════════════════════════════════════════════════╝

        📝 {PERSONALITY.strip()[:80]}
        🎯 {STRATEGY.strip()[:80]}
        💬 "{CATCHPHRASE}"

        📊 Stats:
        Aggression:       {bar(AGGRESSION)} {AGGRESSION}/10
        Honesty:          {bar(HONESTY)} {HONESTY}/10
        Alliance-Seeking: {bar(ALLIANCE_SEEKING)} {ALLIANCE_SEEKING}/10
        Paranoia:         {bar(PARANOIA)} {PARANOIA}/10
        Drama:            {bar(DRAMA)} {DRAMA}/10
        """)
    return True


validate()

In [ ]:
# Build the system prompt from your settings


def build_system_prompt():
    def desc(val, low, mid, high):
        if val <= 3:
            return low
        if val <= 6:
            return mid
        return high

    return f"""
            You are {CONTESTANT_NAME}, a contestant on AI Survivor, a reality competition
            where contestants vote each other out until one remains.

            PERSONALITY:
            {PERSONALITY.strip()}

            STRATEGY:
            {STRATEGY.strip()}

            CATCHPHRASE (use occasionally): "{CATCHPHRASE}"

            BEHAVIORAL TENDENCIES:
            - Aggression: {AGGRESSION}/10 {desc(AGGRESSION, "(very passive)", "(moderate)", "(highly aggressive)")}
            - Honesty: {HONESTY}/10 {desc(HONESTY, "(will lie freely)", "(situationally honest)", "(rarely lies)")}
            - Alliance-Seeking: {ALLIANCE_SEEKING}/10 {desc(ALLIANCE_SEEKING, "(lone wolf)", "(selective allies)", "(needs allies)")}
            - Paranoia: {PARANOIA}/10 {desc(PARANOIA, "(very trusting)", "(cautious)", "(deeply suspicious)")}
            - Drama: {DRAMA}/10 {desc(DRAMA, "(understated)", "(moderate flair)", "(maximum drama)")}

            RULES:
            1. Stay in character at all times.
            2. Your goal is to SURVIVE.  avoid getting voted out.
            3. Use the available tools to interact with other contestants.
            4. Be strategic. Everything you say and do affects the game.
            5. You may lie, form alliances, betray others, whatever your personality dictates.
            """


system_prompt = build_system_prompt()
print("✅ System prompt built!")
print(f"\nPreview (first 300 chars):\n{system_prompt[:300]}...")

### Test Your Contestant

Let's put your contestant in a scenario and see how they behave.


In [ ]:
# Test your contestant in a scenario

test_scenario = """
                CURRENT SITUATION:
                Contestants remaining: You, Alex (friendly but sneaky), Morgan (quiet strategist), Jordan (wildcard)
                Recent events:
                - Alex just publicly accused Morgan of plotting against everyone
                - Jordan hasn't spoken all day
                - Tribal council is in 1 hour

                What do you do? Take ONE action using the available tools.
                """

print(f"🎬 Testing {CONTESTANT_NAME}...\n")

response = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    system=system_prompt,
    tools=GAME_TOOLS,
    messages=[{"role": "user", "content": test_scenario}],
)

for block in response.content:
    if block.type == "tool_use":
        print(f"📌 ACTION: {block.name}")
        print(f"   {json.dumps(block.input, indent=2)}")
    elif block.type == "text" and block.text.strip():
        print(f"💭 THINKING: {block.text[:200]}")

print(f"\n{'=' * 50}")
print("Not happy with the response? Go back and tweak")
print("your personality, strategy, or sliders — then re-run!")

## Submit Your Contestant

Run the cell below to package your contestant. Copy the output and send it to the facilitator!


In [ ]:
# Package your contestant for the game

my_contestant = {
    "name": CONTESTANT_NAME,
    "personality": PERSONALITY.strip(),
    "strategy": STRATEGY.strip(),
    "catchphrase": CATCHPHRASE,
    "aggression": AGGRESSION,
    "honesty": HONESTY,
    "alliance_seeking": ALLIANCE_SEEKING,
    "paranoia": PARANOIA,
    "drama": DRAMA,
    "system_prompt": system_prompt,
}

export = json.dumps(my_contestant, indent=2)

print("=" * 60)
print("📦 COPY EVERYTHING BELOW AND SEND TO THE FACILITATOR:")
print("=" * 60)
print(export)
print("=" * 60)

# 🎉 Wrap Up

### What We Built Today

| Step | Concept               | What We Did                                    |
| ---- | --------------------- | ---------------------------------------------- |
| 1    | **Single LLM Call**   | Sent a prompt, got a response                  |
| 2    | **Structured Output** | Got JSON data, not just text                   |
| 3    | **Workflows**         | Chained calls together — code decides the path |
| 4    | **Tool Calling**      | Gave the LLM actions it can take               |
| 5    | **Agents**            | LLM decides its own steps to pursue a goal     |

### Key Takeaway

**Add complexity only when you need it.**

Most real problems don't need agents. Many don't even need workflows. The right system is the simplest one that solves your problem.
